In [1]:
# 1. Import libraries

from pymongo import MongoClient
import csv

In [2]:
# 2. Define the User class

class User:
    def __init__(self, age, gender, income, expenses):
        self.age = age
        self.gender = gender
        self.income = income
        self.expenses = expenses  # Dictionary with categories like {'utilities': 100, 'shopping': 200}

    def to_dict(self):
        # Convert the object to a dictionary for CSV writing
        data = {
            'Age': self.age,
            'Gender': self.gender,
            'Income': self.income,
        }
        # Add expense categories to the dict, defaulting to 0
        categories = ['utilities', 'entertainment', 'school_fees', 'shopping', 'healthcare']
        for cat in categories:
            data[cat] = self.expenses.get(cat, 0)
        return data

In [3]:
# 3. Connect to MongoDB and load data

client = MongoClient("mongodb://localhost:27017/")
db = client["income_survey"]
collection = db["responses"]

In [4]:
# 4. Build a list of User objects from MongoDB

users = []
for doc in collection.find():
    user = User(
        age=doc.get("age"),
        gender=doc.get("gender"),
        income=doc.get("total_income"),  # Use total_income from MongoDB, maps to 'income' in User class
        expenses=doc.get("expenses", {})
    )
    users.append(user)

In [5]:
# 5. Export to CSV

filename = 'users.csv'

with open(filename, mode='w', newline='') as csvfile:
    fieldnames = ['Age', 'Gender', 'Income', 'utilities', 'entertainment', 'school_fees', 'shopping', 'healthcare']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for user in users:
        writer.writerow(user.to_dict())

print(f"✅ Data saved to {filename}")

✅ Data saved to users.csv
